# [LVV-T2232] - M1M3 Integration with SAL

The objective of this test case is to verify the latest M1M3 commands, events, and telemetry defined by the latest version of the XML.  
This test case will exercise the functionality of the M1M3 on the 3rd level of the Summit and meets the following criteria:
- Only requires the most current version of SAL
- Only requires the M1M3 surrogate to be loaded on the cell
- Requires the use of the DDS and the EFD

[LVV-T2232]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2232

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)

In [ ]:
test_case = "LVV-T2232"
test_execution = "LVV-E1285"

---
## LVV-T1996 (1.0) M1M3 DDS Startup Procedure

[LVV-T1996 (1.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1996

In [ ]:
import asyncio
import os
import yaml

import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy import time 
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from datetime import datetime, timedelta

from lsst_efd_client import EfdClient
from lsst.ts import utils, salobj
from lsst.ts.criopy import M1M3FATable
from lsst.ts.observatory.control.maintel.mtcs import MTCS, MTCSUsages
from lsst.ts.observatory.control import RotType

import lsst.sitcom.vandv as vandv

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
index = exec_info.get_index(test_case, test_execution)

start_time = datetime.now()
script = salobj.Controller("Script", index=index)

In [ ]:
await mtcs.set_state(state=salobj.State.DISABLED, components=["mtm1m3"], overrides={"mtm1m3": "Default"})

In [ ]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm1m3"])

In [ ]:
script.log.info(f"{test_case} - {test_execution} - Start")

---
## Telemetry Verification

Verify the MTM1M3_forceActuatorData telemetry data is being published to the EFD with the following parameters:

```
primaryCylinderForce    secondaryCylinderForce    xForce    yForce    zForce
fx                      fy                        fz        mx        my
mz                      timestamp                 forceMagnitude
```

Check [Chronograph - M1M3 Status].

[Chronograph - M1M3 Status]: https://chronograf-tucson-teststand-efd.lsst.codes/sources/1/dashboards/37?refresh=Paused&lower=now%28%29%20-%205m

In [ ]:
if exec_info.loc == "summit":
    client = EfdClient("summit_efd")
elif location == "tucson":
    client = EfdClient("tucson_teststand_efd")
else:
    raise ValueError(
        "Location does not match any valid options {summit|tucson}"
    )

In [ ]:
start = time.Time("2022-06-14T20:20", scale="utc", format="isot")
end = time.Time("2022-06-14T20:30", scale="utc", format="isot")

In [ ]:
df = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [ ]:
df

In [ ]:
df.iloc[0]

---
Verify the `MTM1M3_forceActuatorPressure` telemetry data is being published to the EFD with the following parameters:
- timestamps
- primaryCylinderPullPressures
- primaryCylinderPushPressures
- secondaryCylinderPullPressures
- secondaryCylinderPushPressures

In [ ]:
fap_df = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorPressure", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [ ]:
fap_df

---
Verify the `MTM1M3_inclinometerDatatelemetry` data is being published to the EFD with the following parameters:
- timestamp
- inclinometerAngle

In [ ]:
df_id = await client.select_time_series(
    "lsst.sal.MTM1M3.outerLoopData", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [ ]:
df_id

---
Verify the MTM1M3_pidData telemetry data is being published to the EFD with the following parameters:

    measuredPID
    timestamp
    setpoint
    error
    errorT1
    errorT2
    control
    controlT1
    controlT2

In [ ]:
df_pidData = await client.select_time_series(
    "lsst.sal.MTM1M3.pidData", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [ ]:
df_pidData

In [ ]:
script.log.info("LVV-T12232 - LVV-E1285 - END")

---
## Events Verification
Verification that some events are initially published to the EFD:
 

In [ ]:
start = time.Time("2022-06-14T20:20", scale="utc", format="isot")
end = time.Time("2022-06-14T20:30", scale="utc", format="isot")

In [ ]:
events = ['lsst.sal.MTM1M3.logevent_hardpointActuatorInfo',
          'lsst.sal.MTM1M3.logevent_hardpointMonitorInfo', 
          'lsst.sal.MTM1M3.logevent_forceActuatorInfo', 
          'lsst.sal.MTM1M3.logevent_forceActuatorState', 
          'lsst.sal.MTM1M3.logevent_hardpointActuatorSettings', 
          'lsst.sal.MTM1M3.logevent_displacementSensorSettings', 
          'lsst.sal.MTM1M3.logevent_pidSettings', 
          'lsst.sal.MTM1M3.logevent_gyroSettings', 
          'lsst.sal.MTM1M3.logevent_inclinometerSettings', 
          'lsst.sal.MTM1M3.logevent_positionControllerSettings', 
          'lsst.sal.MTM1M3.logevent_forceActuatorSettings', 
          'lsst.sal.MTM1M3.logevent_accelerometerSettings', 
          'lsst.sal.MTM1M3.logevent_interlockStatus', 
          'lsst.sal.MTM1M3.logevent_powerSupplyStatus' ]

In [ ]:
for logevent in events:
    #print(f'{logevent} between {start} and  {end}') 
    df = await client.select_time_series(
        logevent, 
        fields="*", 
        start=start.utc, 
        end=end.utc,
    )
    print(logevent.center(120, '*'))
    print(f'\n{df}\n')
    
    df.to_csv(f'./logevent/m1m3/lVV-T2232_{logevent}.csv')

---
### Warning Events
Note: The following steps are meant to verify the warning events are being published to the EFD via SAL, not as a result of an actual warning. 

Check that logevent warnings are being publisht to the EFD. 

In [ ]:
warning_events = ['lsst.sal.MTM1M3.logevent_airSupplyWarning',
            'lsst.sal.MTM1M3.logevent_ilcWarning',
            'lsst.sal.MTM1M3.logevent_forceActuatorWarning',
            'lsst.sal.MTM1M3.logevent_interlockWarning',
            'lsst.sal.MTM1M3.logevent_displacementSensorWarning',
            'lsst.sal.MTM1M3.logevent_inclinometerSensorWarning',
            'lsst.sal.MTM1M3.logevent_accelerometerWarning',
            'lsst.sal.MTM1M3.logevent_forceSetpointWarning',
            'lsst.sal.MTM1M3.logevent_gyroWarning',
            'lsst.sal.MTM1M3.logevent_forceActuatorForceWarning']
            

In [ ]:
for warning in warning_events:
    #print(f'{warning} between {start} and  {end}') 
    df = await client.select_time_series(
        warning, 
        fields="*", 
        start=start, 
        end=end,
    )
    print(warning.center(120, '*'))
    print(f'\n{df}\n')
    
    df.to_csv(f'./logevent/m1m3/lVV-T2232_{warning}.csv')

---
### Preclipped Events Verification
Note: The following steps are meant to verify the preclipped related events are being published to the EFD via SAL at startup.

In [ ]:
preclipped_events = ['lsst.sal.MTM1M3.logevent_preclippedStaticForces',
            'lsst.sal.MTM1M3.logevent_preclippedElevationForces',
            'lsst.sal.MTM1M3.logevent_preclippedAzimuthForces',
            'lsst.sal.MTM1M3.logevent_preclippedThermalForces',
            'lsst.sal.MTM1M3.logevent_preclippedActiveOpticForces',
            'lsst.sal.MTM1M3.logevent_preclippedBalanceForces',
            'lsst.sal.MTM1M3.logevent_preclippedVelocityForces',
            'lsst.sal.MTM1M3.logevent_preclippedAccelerationForces',
            'lsst.sal.MTM1M3.logevent_preclippedOffsetForces',
            'lsst.sal.MTM1M3.logevent_preclippedForces',
            'lsst.sal.MTM1M3.logevent_preclippedCylinderForces']
            

In [ ]:
for events in preclipped_events:
    #print(f'{warning} between {start} and  {end}') 
    df = await client.select_time_series(
        events, 
        fields="*", 
        start=start, 
        end=end,
    )
    print(events.center(120, '*'))
    print(f'\n{df}\n')
    
    df.to_csv(f'./logevent/m1m3/lVV-T2232_{warning}.csv')

---
## 61 - Engineering Mode Test

While the M1M3 is enabled and in the PARKED state, send an `MTM1M3_command_enterEngineering` command. 

`PARKED` means that it is not raised.

In [ ]:
await mtcs.rem.mtm1m3.cmd_enterEngineering.set_start()

In [ ]:
from lsst.ts.idl.enums.MTM1M3 import DetailedState

m1m3_dstate = mtcs.rem.mtm1m3.evt_detailedState.get()

print(DetailedState.PARKEDENGINEERING == m1m3_dstate.detailedState)

---
With the system in the ParkedEngineering state and the M1M3 cell lights off, send an MTM1M3_command_turnLightsOn command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_turnLightsOn.set_start()

In [ ]:
print( mtcs.rem.mtm1m3.evt_cellLightStatus.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_cellLightWarning.get() )

---
With the system in the ParkedEngineering state and the M1M3 cell lights off, send an MTM1M3_command_turnLightsOff command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_turnLightsOff.set_start()

In [ ]:
print( mtcs.rem.mtm1m3.evt_cellLightStatus.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_cellLightWarning.get() )

---
With the system in the ParkedEngineering state and the telescope is not moving, send an MTM1M3_command_setAirSlewFlag command to open the booster valves.

In [ ]:
print( mtcs.rem.mtm1m3.evt_forceActuatorState.get() )

In [ ]:
await mtcs.rem.mtm1m3.cmd_setAirSlewFlag.set_start()

In [ ]:
print( mtcs.rem.mtm1m3.evt_forceActuatorState.get() )

---
In the ParkedEngineering state, send an MTM1M3_command_testHardpoint command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_killHardpointTest.set_start(hardpointActuator=1)

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointTestStatus.get() )

In [ ]:
await mtcs.rem.mtm1m3.cmd_testHardpoint.set_start(hardpointActuator=1)

Verify the appropriate events are published

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointActuatorWarning.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointMonitorWarning.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointTestStatus.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointActuatorState.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointMonitorState.get() )

---
killHardpointTest command


In [ ]:
await mtcs.rem.mtm1m3.cmd_testHardpoint.set_start(hardpointActuator=1)
await asyncio.sleep(5)
await mtcs.rem.mtm1m3.cmd_killHardpointTest.set_start(hardpointActuator=1)

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointActuatorWarning.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointMonitorWarning.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointTestStatus.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointActuatorState.get() )

In [ ]:
print( mtcs.rem.mtm1m3.evt_hardpointMonitorState.get() )

---
In the enabled state, send an MTM1M3_command_moveHardpointActuators command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_moveHardpointActuators.set_start(steps=[1000]*6)

---
While the M1M3 is still in motion, send an MTM1M3_command_stopHardpointMotion command.

In [ ]:
# The range of the actuators is up to 64k 
await mtcs.rem.mtm1m3.cmd_moveHardpointActuators.set_start(steps=[10000]*6)
await asyncio.sleep(3)
await mtcs.rem.mtm1m3.cmd_stopHardpointMotion.set_start()

In [ ]:
# Turned the air on again since I turned it off before by accident
await mtcs.rem.mtm1m3.cmd_turnAirOn.set_start()

---
In the enabled state, send an MTM1M3_command_moveHardpointActuators command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_moveHardpointActuators.set_start(steps=[-10000]*6)

---
## Enabled Force Actuator Test
Verify the MTM1M3_logevent_enabledForceActuators event is published

In [ ]:
await mtcs.rem.mtm1m3.cmd_disableForceActuator.set_start(actuatorId=225)

In [ ]:
print( mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled[60] )

In [ ]:
await mtcs.rem.mtm1m3.cmd_enableForceActuator.set_start(actuatorId=225)

In [ ]:
print( mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled[60] )

In the enabled state of the Engineering mode, disable at least two force actuators by sending an MTM1M3_command_disableForceActuator command one at a time.

Note: Any Force actuators can be disabled as long as they're not near neighbors or next to near neighbors. The actuators will also need to be disabled one at a time. For example, use force actuators 208 (index 44) and 417 (index 130).

In [ ]:
await mtcs.rem.mtm1m3.cmd_enableAllForceActuators.set_start()

In [ ]:
await mtcs.rem.mtm1m3.cmd_disableForceActuator.set_start(actuatorId=208)
print( mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled[44] )

In [ ]:
print( mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled[44] )

In [ ]:
await mtcs.rem.mtm1m3.cmd_disableForceActuator.set_start(actuatorId=417)
await asyncio.sleep(0.1)  # This is only to deal with the async behavior
print( mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled[130] )

---
In the enabled state of the Engineering mode, send an MTM1M3_command_enableAllForceActuators command.


In [ ]:
await mtcs.rem.mtm1m3.cmd_enableAllForceActuators.set_start()
await asyncio.sleep(0.1)  # This is only to deal with the async behavior
print("Actuator 208 enabled? ", mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled[44] )
print("Actuator 417 enabled? ", mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled[130] )
print("All actuators enabled?", all(mtcs.rem.mtm1m3.evt_enabledForceActuators.get().forceActuatorEnabled) )

---
## Raise M1M3
With the M1M3 in the enabled state, send an MTM1M3_command_raiseM1M3 command. However, before the mirror is fully raised, send an MTM1M3_command_abortRaiseM1M3command

In [ ]:
await mtcs.rem.mtm1m3.cmd_raiseM1M3.set_start()
await asyncio.sleep(10)
await mtcs.rem.mtm1m3.cmd_abortRaiseM1M3.set_start()

---
Send an MTM1M3_command_raiseM1M3 command and allow the M1M3 to be fully raised.

In [ ]:
await mtcs.rem.mtm1m3.cmd_raiseM1M3.set_start()

---
With the M1M3 fully raised and the system in the ActiveEngineering state, send an MTM1M3_command_lowerM1M3 command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_lowerM1M3.set_start()

---
With the M1M3 lowered and the system still in the ParkedEngineering state, send an MTM1M3_command_raiseM1M3 command. While the mirror begins to raise, send an MTM1M3_command_disableHardpointChase command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_raiseM1M3.set_start()
await asyncio.sleep(10)
await mtcs.rem.mtm1m3.cmd_disableHardpointChase.set_start()

We expect the system to go into FAULT

In [ ]:
print( mtcs.rem.mtm1m3.evt_detailedState.get() )

To recover the system, use the following steps:

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.DISABLED, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm1m3"])

---

Repeat the same sequence of steps as above but we do not expect a fault as the MTM1M3_command_enableHardpointChase command is sent before the M1M3 is fully raised

In [ ]:
await mtcs.rem.mtm1m3.cmd_raiseM1M3.set_start()
await asyncio.sleep(10)
await mtcs.rem.mtm1m3.cmd_disableHardpointChase.set_start()
await asyncio.sleep(10)
await mtcs.rem.mtm1m3.cmd_enableHardpointChase.set_start()

---
In the ACTIVEENGINEERING state, send an MTM1M3_command_updatePID command with the following parameters:
- pid: 2 (any number 1-6)
- timestep: 0.03s (default is 0.02)
- p: 0.02
- i: 3.0
- d: 0
- n: 0

In [ ]:
await mtcs.rem.mtm1m3.cmd_updatePID.set_start(
    pid=2, timestep=0.03, p=0.02, i=3.0, d=0, n=0)

In [ ]:
print( mtcs.rem.mtm1m3.evt_forceActuatorState.get().balanceForcesApplied )

In [ ]:
print( mtcs.rem.mtm1m3.evt_pidInfo.get() )

---
In the ACTIVEENGINEERING state, send an MTM1M3_command_resetPID command for 2

In [ ]:
await mtcs.rem.mtm1m3.cmd_resetPID.set_start(pid=2)

In [ ]:
print( mtcs.rem.mtm1m3.evt_forceActuatorState.get().balanceForcesApplied )

In [ ]:
print( mtcs.rem.mtm1m3.evt_pidInfo.get() )

---
In the ACTIVEENGINEERING state, send an MTM1M3_command_runMirrorForceProfile command of (10,10,10,10,10,10)

In [ ]:
await mtcs.rem.mtm1m3.cmd_runMirrorForceProfile.set_start(
    xForce=[0]*1000, 
    yForce=10, zForce=10, xMoment=10, yMoment=10, zMoment=10)

---
Send an MTM1M3_command_abortProfile command.

In [ ]:
await mtcs.rem.mtm1m3.cmd_abortProfile.set_start()

---
Verify the MTM1M3_logevent_forceActuatorBumpTestStatus event is published to the EFD.

_LVV-T229 is called to test here to verify the MTM1M3_command_forceActuatorBumpTest command, MTM1M3_command_killForceActuatorBumpTest command and the MTM1M3_logevent_forceActuatorBumpTestStatus event; The test case itself cites a script to use_

In [ ]:
print( mtcs.rem.mtm1m3.evt_forceActuatorBumpTestStatus.get() )
t = time.Time(mtcs.rem.mtm1m3.evt_forceActuatorBumpTestStatus.get().private_sndStamp, format="unix", scale="tai")
t.format = "isot"
print(t)

---

In [ ]:
await mtcs.rem.mtm1m3.cmd_translateM1M3.set_start(
    xTranslation=3, yTranslation=3, zTranslation=3, xRotation=0.5, yRotation=0.5, zRotation=0.5)

In [ ]:
fad_df = await client.select_time_series(
    "lsst.sal.MTM1M3.forceActuatorData", 
    fields="*", 
    start=start.utc, 
    end=end.utc,
)

In [ ]:
fad_df

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
script.log.info(f"{test_case} - {test_execution} - End")

---
_Step 57 is to apply offset forces, but we need Petr's input as it was expected to apply a vector of forces_

await mtcs.rem.mtm1m3.cmd_applyOffsetForces.set_start(_xForces= ,yForces= ,zForces=_)

In [ ]:
await mtcs.rem.mtm1m3.cmd_clearOffsetForces.set_start()

In [ ]:
await mtscs.rem.mtm1m3.cmd_applyOffsetForces(
    xForces=2000, yForces=0, zForces=0)

In [ ]:
print( mtcs.rem.mtm1m3.evt_commandRejectionWarning.get() )

## Absolute Position Set

In the ACTIVEENGINEERING state, send an MTM1M3_command_positionM1M3 command of (3mm, 3mm, 3mm, 0.1deg, 0.1deg, 0.1deg) 

In [ ]:
await mtcs.rem.mtm1m3.cmd_clearOffsetForces.set_start(
    xPosition=3,yPosition=3,zPosition=3,xRotation=0.1,yRotation=0.1,zRotation=0.1)

## Offset Forces

In the enabled, ACTIVEENGINEERING state, send an MTM1M3_command_applyOffsetForcesByMirrorForce command of (100N, 100N, 100N, 100Nm, 100Nm, 100Nm).

In [ ]:
await mtcs.rem.mtm1m3.cmd_applyOffsetForcesByMirrorForce.set_start(
    xForce=100,yForce=100,zForce=100,xMoment=100,yMoment=100,zMoment=100)

## Air Supply Test

Lower the mirror before turning the air off and on.

In [ ]:
await mtcs.rem.mtm1m3.cmd_lowerM1M3.set_start()

In [ ]:
await mtcs.rem.mtm1m3.cmd_turnAirOff.set_start()

In [ ]:
print( mtcs.rem.mtm1m3.evt_airSupplyStatus.get() )
print( mtcs.rem.mtm1m3.evt_airSupplyWarning.get() )

In [ ]:
await mtcs.rem.mtm1m3.cmd_turnAirOn.set_start()

In [ ]:
print( mtcs.rem.mtm1m3.evt_airSupplyStatus.get() )
print( mtcs.rem.mtm1m3.evt_airSupplyWarning.get() )

## Power On/Off

In the PARKEDENGINEERING state, send an MTM1M3_command_turnPowerOff and MTM1M3_command_turnPowerOn command.

**Need Petr's input on which buses we can turn off that aren't responsible for the ILC's**

In [ ]:
await mtcs.rem.mtm1m3.cmd_turnPowerOff.set_start(
    turnPowerNetworkAOff=true,
    turnPowerNetworkBOff=false,
    turnPowerNetworkCOff=false,
    turnPowerNetworkDOff=false,
    turnAuxPowerNetworkAOff=true, 
    turnAuxPowerNetworkBOff=false,
    turnAuxPowerNetworkCOff=false,
    turnAuxPowerNetworkDOff=false)

In [ ]:
print( mtcs.rem.mtm1m3.evt_powerStatus.get() )
print( mtcs.rem.mtm1m3.evt_powerWarning.get() )

In [ ]:
await mtcs.rem.mtm1m3.cmd_turnPowerOff.set_start(
    turnPowerNetworkAOff=true,
    turnPowerNetworkBOff=false,
    turnPowerNetworkCOff=false,
    turnPowerNetworkDOff=false,
    turnAuxPowerNetworkAOff=true, 
    turnAuxPowerNetworkBOff=false,
    turnAuxPowerNetworkCOff=false,
    turnAuxPowerNetworkDOff=false)

In [ ]:
print( mtcs.rem.mtm1m3.evt_powerStatus.get() )
print( mtcs.rem.mtm1m3.evt_powerWarning.get() )

In [ ]:
await mtcs.rem.mtm1m3.cmd_exitEngineering.set_start()

## Aberration Force Test

Send the MTM1M3_command_raiseMTM1M3 command and then record the data of the aberration forces

**There are no aberration force events still so we will need to read the actuator data and compare to the initial data so at this point there is not much to be done through the notebook**

In [ ]:
await mtcs.rem.mtm1m3.cmd_raiseM1M3.set_start()

## Active Optic Forces Test

Apply and clear Active Optic Forces

_what is an appropriate parameter to pass here_

In [ ]:
print( mtcs.rem.mtm1m3.evt_appliedActiveOpticForces.get() )

**Update this step**

await mtcs.rem.mtm1m3.cmd_applyActiveOpticForces.set_start(
    zForces=)

In [ ]:
print( mtcs.rem.mtm1m3.evt_appliedActiveOpticForces.get() )

In [ ]:
await mtcs.rem.mtm1m3.cmd_clearActiveOpticForces.set_start()

Verify an unsafe command of 2000N is rejected 

In [ ]:
await mtcs.rem.mtm1m3.cmd_applyActiveOpticForces.set_start(
    zForces=2000)

In [ ]:
print( mtcs.rem.mtm1m3.evt_preclippedActiveOpticForces.get() )

## Panic Test

Send a panic command to verify the system acts as expected

Panic the mirror while it's lowered

In [ ]:
await mtcs.rem.mtm1m3.cmd_panic.set_start()

In [ ]:
print( mtcs.rem.mtm1m3.evt_detailedState.get() )

Recover the system

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.DISABLED, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm1m3"])

Raise the mirror and send the panic command

In [ ]:
await mtcs.rem.mtm1m3.cmd_raiseM1M3.set_start()

In [ ]:
await mtcs.rem.mtm1m3.cmd_panic.set_start()

In [ ]:
print( mtcs.rem.mtm1m3.evt_detailedState.get() )

Recover the system

In [ ]:
await mtcs.set_state(state=salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.DISABLED, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(state=salobj.State.ENABLED, components=["mtm1m3"])

_At this point the mirror should be lowered and the test can end_

In [ ]:
print( mtcs.rem.mtm1m3.evt_forceActuatorBumpTestStatus.get() )
t = time.Time(mtcs.rem.mtm1m3.evt_forceActuatorBumpTestStatus.get().private_sndStamp, format="unix", scale="tai")
t.format = "isot"
print(t)

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
script.log.info(f"{test_case} - {test_execution} - End")